In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-stat-circo/indic-stat-circonscriptions-legislatives-2022.xlsx
/kaggle/input/resultats-definitifs-par-bureau-de-vote/resultats-definitifs-par-bureau-de-vote.xlsx
/kaggle/input/resultats-definitifs-par-circonscription/resultats-definitifs-par-circonscription.xlsx
/kaggle/input/stats-def/stats_def.xlsx
/kaggle/input/contours-france-entiere-latest-v2/contours-france-entiere-latest-v2.geojson
/kaggle/input/resultats-definitifs-par-departement/resultats-definitifs-par-departement.xlsx
/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.shp
/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.shx
/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.prj
/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.dbf
/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.cpg


In [2]:
!pip install -q streamlit
!pip install geopandas
!pip install folium

In [3]:
!rm -rf /kaggle/working/*

In [4]:
import os
import pandas as pd
import numpy as np

import geopandas as gpd
import folium

In [5]:
def new_folder(new_folder_path):
    # Check if the folder exists, if not, create it
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        print(f'Created new folder at {new_folder_path}')
    else:
        print(f'Folder {new_folder_path} already exists')

In [6]:
def save_group_to_csv(df,column_name, output_directory,data_type):   
    groups = df.groupby(column_name)
    for name,group in groups:
        filename = f"{output_directory}/{data_type}_{name}.csv"
        group.to_csv(filename, index=False)
        print(f"Exported {filename}")

In [7]:
def save_row_to_csv(row, output_directory='/kaggle/working/circo/data/'):    
    # Define the CSV file path
    filename = f"{output_directory}/data_{row.id_circo}.csv"
    # Save the row to a CSV file
    row.to_csv(filename)

**0. Départements**

In [8]:
# Define the path for the new folder
dpt_folder_path = '/kaggle/working/dpt/'
new_folder(dpt_folder_path)

data_dpt_folder_path = '/kaggle/working/dpt/data/'
new_folder(data_dpt_folder_path)

Created new folder at /kaggle/working/dpt/
Created new folder at /kaggle/working/dpt/data/


In [9]:
df_dpt_resultats = pd.read_excel('/kaggle/input/resultats-definitifs-par-departement/resultats-definitifs-par-departement.xlsx')
df_dpt_resultats.rename(columns={'Code département': 'codeDepartement'}, inplace=True)
df_dpt_resultats['codeDepartement'] = df_dpt_resultats['codeDepartement'].astype(str)
df_dpt_resultats = df_dpt_resultats.melt(id_vars=['codeDepartement', 'Libellé département', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_dpt_resultats.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)

In [10]:
df_dpt_resultats_test = df_dpt_resultats[df_dpt_resultats['codeDepartement'].isin(['14','43','78'])]
df_dpt_resultats_test

,codeDepartement,Libellé département,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,% Exprimés/inscrits,% Exprimés/votants,Blancs,% Blancs/inscrits,% Blancs/votants,Nuls,% Nuls/inscrits,% Nuls/votants,indicateur,valeur
13,14,Calvados,516074,286206,"55,46%",229868,"44,54%",277933,"53,86%","97,11%",3581,"0,69%","1,25%",4692,"0,91%","1,64%",Numéro de panneau 1,1
43,43,Haute-Loire,182507,104108,"57,04%",78399,"42,96%",99835,"54,70%","95,90%",2056,"1,13%","1,97%",2217,"1,21%","2,13%",Numéro de panneau 1,1
78,78,Yvelines,984280,549783,"55,86%",434497,"44,14%",540712,"54,93%","98,35%",4958,"0,50%","0,90%",4113,"0,42%","0,75%",Numéro de panneau 1,1
120,14,Calvados,516074,286206,"55,46%",229868,"44,54%",277933,"53,86%","97,11%",3581,"0,69%","1,25%",4692,"0,91%","1,64%",Nuance liste 1,LDIV
150,43,Haute-Loire,182507,104108,"57,04%",78399,"42,96%",99835,"54,70%","95,90%",2056,"1,13%","1,97%",2217,"1,21%","2,13%",Nuance liste 1,LDIV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32357,43,Haute-Loire,182507,104108,"57,04%",78399,"42,96%",99835,"54,70%","95,90%",2056,"1,13%","1,97%",2217,"1,21%","2,13%",% Voix/exprimés 38,"0,01%"
32392,78,Yvelines,984280,549783,"55,86%",434497,"44,14%",540712,"54,93%","98,35%",4958,"0,50%","0,90%",4113,"0,42%","0,75%",% Voix/exprimés 38,"0,00%"
32434,14,Calvados,516074,286206,"55,46%",229868,"44,54%",277933,"53,86%","97,11%",3581,"0,69%","1,25%",4692,"0,91%","1,64%",Sièges 38,0
32464,43,Haute-Loire,182507,104108,"57,04%",78399,"42,96%",99835,"54,70%","95,90%",2056,"1,13%","1,97%",2217,"1,21%","2,13%",Sièges 38,0


In [11]:
save_group_to_csv(df_dpt_resultats_test,'codeDepartement','/kaggle/working/dpt/data','resultats')

Exported /kaggle/working/dpt/data/resultats_14.csv
Exported /kaggle/working/dpt/data/resultats_43.csv
Exported /kaggle/working/dpt/data/resultats_78.csv


**1. Circonscriptions législatives**

In [12]:
# Define the path for the new folder
circo_folder_path = '/kaggle/working/circo/'
new_folder(circo_folder_path)

map_circo_folder_path = '/kaggle/working/circo/map/'
new_folder(map_circo_folder_path)

data_circo_folder_path = '/kaggle/working/circo/data/'
new_folder(data_circo_folder_path)

Created new folder at /kaggle/working/circo/
Created new folder at /kaggle/working/circo/map/
Created new folder at /kaggle/working/circo/data/


a. Cartes

In [13]:
# Set filepath
fp_circo = "/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.shp"

# Read file using gpd.read_file()
df_circo = gpd.read_file(fp_circo)
df_circo = df_circo.to_crs(epsg=4326)
df_circo['centroid'] = df_circo.geometry.centroid
print(df_circo.crs)
df_circo.rename(columns={'dep': 'id_dep'}, inplace=True)
df_circo.head()

/tmp/ipykernel_33/717204969.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_circo['centroid'] = df_circo.geometry.centroid


EPSG:4326


,id_circo,id_dep,libelle,geometry,centroid
0,97302,973,Guyane - 2e circonscription,"MULTIPOLYGON (((-54.60236 2.33356, -54.60242 2...",POINT (-53.61144 4.09182)
1,97301,973,Guyane - 1re circonscription,"MULTIPOLYGON (((-51.93691 4.46648, -51.93667 4...",POINT (-52.55558 3.61228)
2,97201,972,Martinique - 1re circonscription,"MULTIPOLYGON (((-61.00495 14.57791, -61.00492 ...",POINT (-60.96320 14.66706)
3,97202,972,Martinique - 2e circonscription,"MULTIPOLYGON (((-61.12889 14.63162, -61.12890 ...",POINT (-61.10999 14.76083)
4,97203,972,Martinique - 3e circonscription,"POLYGON ((-61.03945 14.64265, -61.03940 14.642...",POINT (-61.06897 14.64060)


In [14]:
# Centroid column
df_circo['lat'] = df_circo["centroid"].y
df_circo['lon'] = df_circo["centroid"].x

df_circo['id_circo_len'] = df_circo['id_circo'].str.len()
df_circo['id_circo'] = np.where(df_circo['id_circo_len'] == 5, df_circo['id_circo'], df_circo['id_circo'].astype(str).str[:2] + '0' + df_circo['id_circo'].astype(str).str[2:])

In [15]:
df_circo_test = df_circo[df_circo['id_dep'].isin(['14','43','78'])]
df_circo_test

,id_circo,id_dep,libelle,geometry,centroid,lat,lon,id_circo_len
60,14002,14,Calvados - 2e circonscription,"MULTIPOLYGON (((-0.09779 49.16358, -0.09779 49...",POINT (-0.23039 49.16719),49.167194,-0.230388,4
136,14001,14,Calvados - 1re circonscription,"MULTIPOLYGON (((-0.50179 49.16021, -0.50179 49...",POINT (-0.50910 49.18438),49.184385,-0.509097,4
158,14003,14,Calvados - 3e circonscription,"MULTIPOLYGON (((-0.41112 48.91971, -0.41112 48...",POINT (-0.02012 49.01783),49.017830,-0.020119,4
176,14006,14,Calvados - 6e circonscription,"MULTIPOLYGON (((-0.94426 48.96699, -0.91508 48...",POINT (-0.67385 48.95874),48.958738,-0.673849,4
270,78001,78,Yvelines - 1re circonscription,"MULTIPOLYGON (((2.00891 48.76789, 2.00891 48.7...",POINT (2.08090 48.78946),48.789456,2.080897,4
272,78002,78,Yvelines - 2e circonscription,"MULTIPOLYGON (((1.92481 48.66980, 1.92481 48.6...",POINT (2.05369 48.72890),48.728905,2.053692,4
407,14005,14,Calvados - 5e circonscription,"MULTIPOLYGON (((-0.92435 49.22818, -0.92435 49...",POINT (-0.77003 49.25623),49.256234,-0.770033,4
408,14004,14,Calvados - 4e circonscription,"MULTIPOLYGON (((0.30604 49.11147, 0.29713 49.1...",POINT (0.10671 49.26170),49.261701,0.106712,4
456,78004,78,Yvelines - 4e circonscription,"MULTIPOLYGON (((2.12436 48.87438, 2.12436 48.8...",POINT (2.13774 48.88680),48.886797,2.137738,4
470,78003,78,Yvelines - 3e circonscription,"MULTIPOLYGON (((2.13101 48.81487, 2.11428 48.8...",POINT (2.03799 48.84967),48.849673,2.037992,4


In [16]:
groups = df_circo_test.groupby('id_dep')
for name,group in groups:
    m = folium.Map(location=[48.858885,2.34694], zoom_start=6, tiles="CartoDB positron")
    for _, r in group.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
        folium.Popup(r["libelle"]+ " " + r["id_circo"]).add_to(geo_j)
        geo_j.add_to(m)
    m.save('/kaggle/working/circo/map/map_' +  r['id_dep'] + '.html')

b. data - https://www.insee.fr/fr/statistiques/6436476?sommaire=6436478

In [17]:
df_circo_stats = pd.read_excel('/kaggle/input/indic-stat-circo/indic-stat-circonscriptions-legislatives-2022.xlsx',
                               sheet_name='indicateurs_circonscriptions',
                               skiprows=7)
df_circo_stats.rename(columns={'circo': 'id_circo'}, inplace=True)
df_circo_stats['id_dep']= df_circo_stats['id_circo'].str[:2]                                        

In [18]:
df_circo_stats_test = df_circo_stats[df_circo_stats['id_circo'].str[:2].isin(['14','43','78'])]
df_circo_stats_test.head()

,id_circo,Nom de la circonscription,Inscrit_22,pop_légal_19,pop_légal_13,tvar_pop,pop_pole_aav,pop_cour_aav,pop_horsaav,pop_urb,...,PPSOC,PIMPOT,acc_ecole,acc_college,acc_lycee,acc_medecin,acc_dentiste,acc_pharmacie,part_eloig,id_dep
60,14001,Calvados - 1re circonscription,71988,112766,111543,0.2,71.9,28.1,0,86.5,...,6.4,-18.1,98.5,72.8,66.8,89.2,80.6,89.2,2.3,14
61,14002,Calvados - 2e circonscription,68045,108854,105416,0.5,85,15,0,90.7,...,8.3,-14.8,97.4,82.1,70.8,95.7,85.4,94.7,1.1,14
62,14003,Calvados - 3e circonscription,79595,106852,109296,-0.4,18.8,64.9,16.3,27.9,...,6.7,-13.8,79.4,53.7,35.4,61.4,54.9,60.6,30.5,14
63,14004,Calvados - 4e circonscription,104420,125987,124737,0.2,32.6,64.1,3.4,56.5,...,4.6,-17.3,84.8,38.6,17,72.2,51.5,66.3,12.1,14
64,14005,Calvados - 5e circonscription,92374,112916,112476,0.1,14.7,77.6,7.7,44.2,...,4.5,-16.5,76.4,30.5,19.2,59.2,35.4,56.3,19.3,14


In [19]:
save_group_to_csv(df_circo_stats_test,'id_dep','/kaggle/working/circo/data','stats')

Exported /kaggle/working/circo/data/stats_14.csv
Exported /kaggle/working/circo/data/stats_43.csv
Exported /kaggle/working/circo/data/stats_78.csv


In [20]:
df_circo_resultats = pd.read_excel('/kaggle/input/resultats-definitifs-par-circonscription/resultats-definitifs-par-circonscription.xlsx')

df_circo_resultats.head()

,Code département,Libellé département,Code circonscription législative,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,% Voix/exprimés 37,Sièges 37,Numéro de panneau 38,Nuance liste 38,Libellé abrégé de liste 38,Libellé de liste 38,Voix 38,% Voix/inscrits 38,% Voix/exprimés 38,Sièges 38
0,01,Ain,0101,1ère circonscription,86824,48408,"55,75%",38416,"44,25%",46887,...,"0,02%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,2,"0,00%","0,00%",0
1,01,Ain,0102,2ème circonscription,102088,58219,"57,03%",43869,"42,97%",56962,...,"0,04%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,2,"0,00%","0,00%",0
2,01,Ain,0103,3ème circonscription,85884,42940,"50,00%",42944,"50,00%",41880,...,"0,05%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
3,01,Ain,0104,4ème circonscription,96186,53221,"55,33%",42965,"44,67%",51624,...,"0,02%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
4,01,Ain,0105,5ème circonscription,78235,41472,"53,01%",36763,"46,99%",40170,...,"0,05%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0


In [21]:
df_circo_resultats.rename(columns={'Code département': 'codeDepartement'}, inplace=True)
df_circo_resultats['codeDepartement'] = df_circo_resultats['codeDepartement'].astype(str)
df_circo_resultats['Code circonscription législative'] = df_circo_resultats['Code circonscription législative'].astype(str)

df_circo_resultats.rename(columns={'codeDepartement': 'id_dep', 'Code circonscription législative': 'codeCirconscription'}, inplace=True)


df_circo_resultats['id_circo'] = np.where(df_circo_resultats['id_dep'].str.len() == 3,
                             df_circo_resultats['id_dep'] + df_circo_resultats['codeCirconscription'].astype(str).str[2:],
                             df_circo_resultats['codeCirconscription'].astype(str).str[:2] + '0' + df_circo_resultats['codeCirconscription'].astype(str).str[2:])

df_circo_resultats = df_circo_resultats.drop(columns=['codeCirconscription']).drop_duplicates()


df_circo_resultats = df_circo_resultats.melt(id_vars=['id_dep', 'Libellé département','id_circo','Libellé circonscription législative', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_circo_resultats.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)

df_circo_resultats_test = df_circo_resultats[df_circo_resultats['id_dep'].isin(['14','43','78'])]
df_circo_resultats_test

,id_dep,Libellé département,id_circo,Libellé circonscription législative,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,% Exprimés/inscrits,% Exprimés/votants,Blancs,% Blancs/inscrits,% Blancs/votants,Nuls,% Nuls/inscrits,% Nuls/votants,indicateur,valeur
59,14,Calvados,14001,1ère circonscription,73133,42510,"58,13%",30623,"41,87%",41668,"56,98%","98,02%",378,"0,52%","0,89%",464,"0,63%","1,09%",Numéro de panneau 1,1
60,14,Calvados,14002,2ème circonscription,68978,37053,"53,72%",31925,"46,28%",36131,"52,38%","97,51%",431,"0,62%","1,16%",491,"0,71%","1,33%",Numéro de panneau 1,1
61,14,Calvados,14003,3ème circonscription,80053,42626,"53,25%",37427,"46,75%",41173,"51,43%","96,59%",568,"0,71%","1,33%",885,"1,11%","2,08%",Numéro de panneau 1,1
62,14,Calvados,14004,4ème circonscription,105044,58533,"55,72%",46511,"44,28%",56999,"54,26%","97,38%",667,"0,63%","1,14%",867,"0,83%","1,48%",Numéro de panneau 1,1
63,14,Calvados,14005,5ème circonscription,93187,52967,"56,84%",40220,"43,16%",51426,"55,19%","97,09%",637,"0,68%","1,20%",904,"0,97%","1,71%",Numéro de panneau 1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175262,78,Yvelines,78008,8ème circonscription,75913,34540,"45,50%",41373,"54,50%",33711,"44,41%","97,60%",444,"0,58%","1,29%",385,"0,51%","1,11%",Sièges 38,0
175263,78,Yvelines,78009,9ème circonscription,94922,49472,"52,12%",45450,"47,88%",48368,"50,96%","97,77%",584,"0,62%","1,18%",520,"0,55%","1,05%",Sièges 38,0
175264,78,Yvelines,78010,10ème circonscription,93105,54902,"58,97%",38203,"41,03%",53901,"57,89%","98,18%",529,"0,57%","0,96%",472,"0,51%","0,86%",Sièges 38,0
175265,78,Yvelines,78011,11ème circonscription,71218,36172,"50,79%",35046,"49,21%",35464,"49,80%","98,04%",373,"0,52%","1,03%",335,"0,47%","0,93%",Sièges 38,0


In [22]:
save_group_to_csv(df_circo_resultats_test,'id_dep','/kaggle/working/circo/data','resultats')

Exported /kaggle/working/circo/data/resultats_14.csv
Exported /kaggle/working/circo/data/resultats_43.csv
Exported /kaggle/working/circo/data/resultats_78.csv


**2. Bureaux de votes**

In [23]:
# Define the path for the new folder
bv_folder_path = '/kaggle/working/bv/'
new_folder(bv_folder_path)

map_bv_folder_path = '/kaggle/working/bv/map/'
new_folder(map_bv_folder_path)

map_bv_folder_path = '/kaggle/working/bv/data/'
new_folder(map_bv_folder_path)

Created new folder at /kaggle/working/bv/
Created new folder at /kaggle/working/bv/map/
Created new folder at /kaggle/working/bv/data/


a. Map

In [24]:
# Set filepath
fp_bv = "/kaggle/input/contours-france-entiere-latest-v2/contours-france-entiere-latest-v2.geojson"

# Read file using gpd.read_file()
df_bv = gpd.read_file(fp_bv)
df_bv = df_bv.to_crs(epsg=4326)
print(df_bv.crs)
df_bv.head()

EPSG:4326


,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,codeCommune,nomCommune,numeroBureauVote,codeBureauVote,id_bv,geometry
0,01,Ain,0104,4ème circonscription,01001,L'Abergement-Clémenciat,0001,01001_0001,01001_1,"POLYGON ((4.95812 46.15316, 4.95771 46.15288, ..."
1,01,Ain,0105,5ème circonscription,01002,L'Abergement-de-Varey,0001,01002_0001,01002_1,"POLYGON ((5.43001 45.98299, 5.42980 45.98312, ..."
2,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0001,01004_0001,01004_1,"POLYGON ((5.33312 45.95904, 5.33358 45.95987, ..."
3,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0002,01004_0002,01004_2,"MULTIPOLYGON (((5.35287 45.95468, 5.35225 45.9..."
4,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0003,01004_0003,01004_3,"POLYGON ((5.36083 45.94544, 5.36069 45.94557, ..."


In [25]:
df_bv['id_dep'] = np.where(df_bv['codeDepartement'] == 'ZA', '971',
                           np.where(df_bv['codeDepartement'] == 'ZB', '972',
                           np.where(df_bv['codeDepartement'] == 'ZC', '973',
                           np.where(df_bv['codeDepartement'] == 'ZD', '974',
                           np.where(df_bv['codeDepartement'] == 'ZS', '975',
                           np.where(df_bv['codeDepartement'] == 'ZM', '976', df_bv['codeDepartement']))))))


df_bv['id_circo'] = np.where(df_bv['id_dep'].str.len() == 3,
                             df_bv['id_dep'] + df_bv['codeCirconscription'].astype(str).str[2:],
                             df_bv['codeCirconscription'].astype(str).str[:2] + '0' + df_bv['codeCirconscription'].astype(str).str[2:])

df_bv['bv_lib'] = df_bv['nomCommune'].str.cat(df_bv['numeroBureauVote'], sep = ' - ')

df_bv.head()

,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,codeCommune,nomCommune,numeroBureauVote,codeBureauVote,id_bv,geometry,id_dep,id_circo,bv_lib
0,01,Ain,0104,4ème circonscription,01001,L'Abergement-Clémenciat,0001,01001_0001,01001_1,"POLYGON ((4.95812 46.15316, 4.95771 46.15288, ...",01,01004,L'Abergement-Clémenciat - 0001
1,01,Ain,0105,5ème circonscription,01002,L'Abergement-de-Varey,0001,01002_0001,01002_1,"POLYGON ((5.43001 45.98299, 5.42980 45.98312, ...",01,01005,L'Abergement-de-Varey - 0001
2,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0001,01004_0001,01004_1,"POLYGON ((5.33312 45.95904, 5.33358 45.95987, ...",01,01005,Ambérieu-en-Bugey - 0001
3,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0002,01004_0002,01004_2,"MULTIPOLYGON (((5.35287 45.95468, 5.35225 45.9...",01,01005,Ambérieu-en-Bugey - 0002
4,01,Ain,0105,5ème circonscription,01004,Ambérieu-en-Bugey,0003,01004_0003,01004_3,"POLYGON ((5.36083 45.94544, 5.36069 45.94557, ...",01,01005,Ambérieu-en-Bugey - 0003


In [26]:
df_bv_test = df_bv[df_bv['id_dep'].isin(['14','43','78'])]
export_test = df_bv_test[['id_dep','nomDepartement','id_circo','nomCirconscription','codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','bv_lib']]
export_test.to_csv('/kaggle/working/dataset_dpt_circo_bv_test.csv', index=False)

In [27]:
df_bv_test['centroid'] = df_bv_test.geometry.centroid
df_bv_test['lat'] = df_bv_test["centroid"].y
df_bv_test['lon'] = df_bv_test["centroid"].x

df_bv_test.head()

/tmp/ipykernel_33/4205623389.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_bv_test['centroid'] = df_bv_test.geometry.centroid
/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,codeDepartement,nomDepartement,codeCirconscription,nomCirconscription,codeCommune,nomCommune,numeroBureauVote,codeBureauVote,id_bv,geometry,id_dep,id_circo,bv_lib,centroid,lat,lon
7776,14,Calvados,1404,4ème circonscription,14001,Ablon,0001,14001_0001,14001_1,"POLYGON ((0.29683 49.42021, 0.29679 49.41277, ...",14,14004,Ablon - 0001,POINT (0.28710 49.39418),49.394176,0.287097
7777,14,Calvados,1405,5ème circonscription,14003,Agy,0001,14003_0001,14003_1,"POLYGON ((-0.79124 49.22918, -0.79201 49.22974...",14,14005,Agy - 0001,POINT (-0.77897 49.23756),49.237564,-0.778968
7778,14,Calvados,1406,6ème circonscription,14005,Valambray,0001,14005_0001,14005_1,"POLYGON ((-0.16746 49.07795, -0.16808 49.07779...",14,14006,Valambray - 0001,POINT (-0.14437 49.09592),49.095919,-0.144369
7779,14,Calvados,1406,6ème circonscription,14005,Valambray,0002,14005_0002,14005_2,"POLYGON ((-0.20891 49.08056, -0.20896 49.08074...",14,14006,Valambray - 0002,POINT (-0.19051 49.08891),49.088911,-0.190506
7780,14,Calvados,1406,6ème circonscription,14005,Valambray,0003,14005_0003,14005_3,"MULTIPOLYGON (((-0.22639 49.07862, -0.22682 49...",14,14006,Valambray - 0003,POINT (-0.24356 49.08671),49.086706,-0.243563


In [28]:
groups = df_bv_test.groupby('id_circo')
for name,group in groups:
    m = folium.Map(location=[48.858885,2.34694], zoom_start=6, tiles="CartoDB positron")
    for _, r in group.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "white"})
        folium.Popup(r["bv_lib"]).add_to(geo_j)
        geo_j.add_to(m)
    m.save('/kaggle/working/bv/map/map_' +  r['id_circo'] + '.html')

b. Résultats Europénnes

In [29]:
df_bv_resultats = pd.read_excel('/kaggle/input/resultats-definitifs-par-bureau-de-vote/resultats-definitifs-par-bureau-de-vote.xlsx')
df_bv_resultats.head()

,Code localisation,Libellé localisation,Code département,Libellé département,Code commune,Libellé commune,Code BV,Inscrits,Votants,% Votants,...,% Voix/exprimés 37,Sièges 37,Numéro de panneau 38,Nuance liste 38,Libellé abrégé de liste 38,Libellé de liste 38,Voix 38,% Voix/inscrits 38,% Voix/exprimés 38,Sièges 38
0,FE,France entière,01,Ain,01001,L'Abergement-Clémenciat,0001,662,373,"56,34%",...,"0,00%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
1,FE,France entière,01,Ain,01002,L'Abergement-de-Varey,0001,228,156,"68,42%",...,"0,00%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
2,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0001,1082,522,"48,24%",...,"0,00%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
3,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0002,1077,566,"52,55%",...,"0,36%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0
4,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0003,1123,640,"56,99%",...,"0,00%",0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0


In [30]:
df_bv_resultats.rename(columns={'Code département': 'id_dep','Code commune': 'codeCommune','Code BV': 'codeBV'}, inplace=True)
df_bv_resultats['codeCommune'] = df_bv_resultats['codeCommune'].astype(str)
df_bv_resultats['codeBV'] = df_bv_resultats['codeBV'].astype(str)

df_bv_resultats['codeBureauVote'] = df_bv_resultats['codeCommune'].str.cat(df_bv_resultats['codeBV'], sep = '_')

df_bv_resultats.head()


,Code localisation,Libellé localisation,id_dep,Libellé département,codeCommune,Libellé commune,codeBV,Inscrits,Votants,% Votants,...,Sièges 37,Numéro de panneau 38,Nuance liste 38,Libellé abrégé de liste 38,Libellé de liste 38,Voix 38,% Voix/inscrits 38,% Voix/exprimés 38,Sièges 38,codeBureauVote
0,FE,France entière,01,Ain,01001,L'Abergement-Clémenciat,0001,662,373,"56,34%",...,0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01001_0001
1,FE,France entière,01,Ain,01002,L'Abergement-de-Varey,0001,228,156,"68,42%",...,0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01002_0001
2,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0001,1082,522,"48,24%",...,0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0001
3,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0002,1077,566,"52,55%",...,0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0002
4,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0003,1123,640,"56,99%",...,0,38,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0003


In [31]:
df_bv_resultats = pd.merge(df_bv_resultats, df_bv[['codeBureauVote','id_circo','bv_lib']], on='codeBureauVote', how='left')
df_bv_resultats = df_bv_resultats.drop_duplicates()
df_bv_resultats

,Code localisation,Libellé localisation,id_dep,Libellé département,codeCommune,Libellé commune,codeBV,Inscrits,Votants,% Votants,...,Nuance liste 38,Libellé abrégé de liste 38,Libellé de liste 38,Voix 38,% Voix/inscrits 38,% Voix/exprimés 38,Sièges 38,codeBureauVote,id_circo,bv_lib
0,FE,France entière,01,Ain,01001,L'Abergement-Clémenciat,0001,662,373,"56,34%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01001_0001,01004,L'Abergement-Clémenciat - 0001
1,FE,France entière,01,Ain,01002,L'Abergement-de-Varey,0001,228,156,"68,42%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01002_0001,01005,L'Abergement-de-Varey - 0001
2,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0001,1082,522,"48,24%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0001,01005,Ambérieu-en-Bugey - 0001
3,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0002,1077,566,"52,55%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0002,01005,Ambérieu-en-Bugey - 0002
4,FE,France entière,01,Ain,01004,Ambérieu-en-Bugey,0003,1123,640,"56,99%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,01004_0003,01005,Ambérieu-en-Bugey - 0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70294,FE,France entière,ZZ,Français établis hors de France,ZZ235,Bahamas (Nassau),0001,161,37,"22,98%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,ZZ235_0001,NaN,NaN
70295,FE,France entière,ZZ,Français établis hors de France,ZZ236,Astana,0001,90,30,"33,33%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,ZZ236_0001,NaN,NaN
70296,FE,France entière,ZZ,Français établis hors de France,ZZ237,Mossoul,0001,1,0,"0,00%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%",0.00%,0,ZZ237_0001,NaN,NaN
70297,FE,France entière,ZZ,Français établis hors de France,ZZ238,Florence,0001,3621,317,"8,75%",...,LDIV,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,LIBERTÉ DÉMOCRATIQUE FRANÇAISE,0,"0,00%","0,00%",0,ZZ238_0001,NaN,NaN


In [32]:
df_bv_resultats_test = df_bv_resultats[df_bv_resultats['id_dep'].isin(['14','43','78'])]
df_bv_resultats_test = df_bv_resultats_test.melt(id_vars=['Code localisation','Libellé localisation','id_dep','Libellé département','id_circo','codeCommune','Libellé commune','codeBV','codeBureauVote','bv_lib', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_bv_resultats_test.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)
df_bv_resultats_test

,Code localisation,Libellé localisation,id_dep,Libellé département,id_circo,codeCommune,Libellé commune,codeBV,codeBureauVote,bv_lib,...,% Exprimés/inscrits,% Exprimés/votants,Blancs,% Blancs/inscrits,% Blancs/votants,Nuls,% Nuls/inscrits,% Nuls/votants,indicateur,valeur
0,FE,France entière,14,Calvados,14004,14001,Ablon,0001,14001_0001,Ablon - 0001,...,"54,30%","97,22%",0,"0,00%","0,00%",15,"1,55%","2,78%",Numéro de panneau 1,1
1,FE,France entière,14,Calvados,14005,14003,Agy,0001,14003_0001,Agy - 0001,...,"61,44%","96,03%",0,"0,00%","0,00%",6,"2,54%","3,97%",Numéro de panneau 1,1
2,FE,France entière,14,Calvados,14006,14005,Valambray,0001,14005_0001,Valambray - 0001,...,"50,19%","97,36%",2,"0,39%","0,75%",5,"0,97%","1,89%",Numéro de panneau 1,1
3,FE,France entière,14,Calvados,14006,14005,Valambray,0002,14005_0002,Valambray - 0002,...,"60,90%","95,86%",3,"1,13%","1,78%",4,"1,50%","2,37%",Numéro de panneau 1,1
4,FE,France entière,14,Calvados,14006,14005,Valambray,0003,14005_0003,Valambray - 0003,...,"46,05%","94,59%",1,"1,32%","2,70%",1,"1,32%","2,70%",Numéro de panneau 1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688859,FE,France entière,78,Yvelines,78002,78688,Voisins-le-Bretonneux,0004,78688_0004,Voisins-le-Bretonneux - 0004,...,"62,21%","97,67%",9,"0,96%","1,50%",5,"0,53%","0,83%",Sièges 38,0
688860,FE,France entière,78,Yvelines,78002,78688,Voisins-le-Bretonneux,0005,78688_0005,Voisins-le-Bretonneux - 0005,...,"57,27%","98,71%",5,"0,37%","0,64%",5,"0,37%","0,64%",Sièges 38,0
688861,FE,France entière,78,Yvelines,78002,78688,Voisins-le-Bretonneux,0006,78688_0006,Voisins-le-Bretonneux - 0006,...,"67,60%","99,50%",2,"0,17%","0,25%",2,"0,17%","0,25%",Sièges 38,0
688862,FE,France entière,78,Yvelines,78002,78688,Voisins-le-Bretonneux,0007,78688_0007,Voisins-le-Bretonneux - 0007,...,"72,84%","99,02%",3,"0,31%","0,42%",4,"0,41%","0,56%",Sièges 38,0


In [33]:
save_group_to_csv(df_bv_resultats_test,'id_circo','/kaggle/working/bv/data','resultats')

Exported /kaggle/working/bv/data/resultats_14001.csv
Exported /kaggle/working/bv/data/resultats_14002.csv
Exported /kaggle/working/bv/data/resultats_14003.csv
Exported /kaggle/working/bv/data/resultats_14004.csv
Exported /kaggle/working/bv/data/resultats_14005.csv
Exported /kaggle/working/bv/data/resultats_14006.csv
Exported /kaggle/working/bv/data/resultats_43001.csv
Exported /kaggle/working/bv/data/resultats_43002.csv
Exported /kaggle/working/bv/data/resultats_78001.csv
Exported /kaggle/working/bv/data/resultats_78002.csv
Exported /kaggle/working/bv/data/resultats_78003.csv
Exported /kaggle/working/bv/data/resultats_78004.csv
Exported /kaggle/working/bv/data/resultats_78005.csv
Exported /kaggle/working/bv/data/resultats_78006.csv
Exported /kaggle/working/bv/data/resultats_78007.csv
Exported /kaggle/working/bv/data/resultats_78008.csv
Exported /kaggle/working/bv/data/resultats_78009.csv
Exported /kaggle/working/bv/data/resultats_78010.csv
Exported /kaggle/working/bv/data/resultats_780

In [34]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory_test.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [35]:
zip_dir()

/kaggle/working/directory_test.zip

In [48]:
%%writefile app.py
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import streamlit as st
st.set_page_config(layout="wide")

# Load the HTML file
def read_html_file(filename):
    with open(filename, 'r') as f:
        return f.read()


# Stats definitions
df_stats_def = pd.read_csv('/kaggle/input/stats-def/stats_def.csv',sep=';')
df_stats_def_demo = df_stats_def[(df_stats_def['Catégorie'] == 'Démographie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_acti = df_stats_def[(df_stats_def['Catégorie'] == 'Activité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_qualif = df_stats_def[(df_stats_def['Catégorie'] == 'Qualification') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_secteur = df_stats_def[(df_stats_def['Catégorie'] == 'Secteur') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_log = df_stats_def[(df_stats_def['Catégorie'] == 'Logements') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_fam = df_stats_def[(df_stats_def['Catégorie'] == 'Famille') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_mob = df_stats_def[(df_stats_def['Catégorie'] == 'Mobilité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_rev = df_stats_def[(df_stats_def['Catégorie'] == 'Revenus') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_niv_vie = df_stats_def[(df_stats_def['Catégorie'] == 'Niveau de vie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_autres = df_stats_def[(df_stats_def['Catégorie'] == 'Autres') | (df_stats_def['Variable'] == 'Nom de la circonscription')]

    
# 1.Liste des départements
file_path = '/kaggle/working/dataset_dpt_circo_bv_test.csv'
df = pd.read_csv(file_path,low_memory=False)
df.rename(columns={'codeDepartement': 'id_dep','nomDepartement': 'dep_name'}, inplace=True)
df['id_dep'] = df['id_dep'].astype(str)
df['id_circo'] = df['id_circo'].astype(str)
df['id_bv'] = df['id_bv'].astype(str)

df_dpt = df.drop(columns=['id_circo','nomCirconscription','codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv']).drop_duplicates()
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt.sort_values(by='id_dep',inplace=True)
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt['dep_lib'] = df_dpt['id_dep'].str.cat(df_dpt['dep_name'], sep = ' - ')

dpt = df_dpt['dep_lib'].drop_duplicates().sort_values()
dpt_selected = st.sidebar.selectbox('Sélection du département:', dpt)
# ID dpt
dpt_id_selected = dpt_selected.split(" - ")[0]

#dpt_id_selected = '14'

file_path_dpt_resultats = '/kaggle/working/dpt/data/resultats_' + dpt_id_selected + '.csv'
dpt_resultats = pd.read_csv(file_path_dpt_resultats,low_memory=False)

# a. Résultats - Stats descriptives - DPT
dpt_resultats_overview = dpt_resultats[['Libellé département', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
dpt_resultats_overview = dpt_resultats_overview.drop_duplicates()

data_container = st.container()
with data_container:
    st.write("Département - Elections européennes:")
    st.dataframe(dpt_resultats_overview,hide_index=True)

# b. Résultats (top10) - DPT
dpt_resultats_details = dpt_resultats[['indicateur','valeur']]
dpt_resultats_details['id_liste'] = dpt_resultats_details['indicateur'].str[-2:]
dpt_resultats_details['id_liste'] = dpt_resultats_details['id_liste'].str.strip()

dpt_resultats_details['indicateur'] = dpt_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
dpt_resultats_details['indicateur']= dpt_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

dpt_resultats_details = dpt_resultats_details.pivot(index ='id_liste', columns='indicateur', values='valeur')
dpt_resultats_details = pd.DataFrame(dpt_resultats_details.to_records())
dpt_resultats_details["Voix"] = dpt_resultats_details["Voix"].astype(int)
dpt_resultats_details = dpt_resultats_details.nlargest(10, 'Voix')
dpt_resultats_details = dpt_resultats_details[['Libellé de liste','Nuance liste', 'Voix','% Voix/exprimés','% Voix/inscrits']]   

data_container2 = st.container()
with data_container2:
    st.write("Département - Résultats européennes (top10):")
    st.dataframe(dpt_resultats_details,hide_index=True)

    
# 2.Circonscriptions du département sélectionné

# a.Carte
#Read the HTML content from the file
html_content = read_html_file('/kaggle/working/circo/map/map_' + dpt_id_selected + '.html')
# Display the HTML content in Streamlit
map_container1 = st.container()

with map_container1:
    st.write("Circonscriptions:")
    st.components.v1.html(html_content,height=500)

# b.Stats des circonscriptions du département sélectionné
file_path_circo_stats = '/kaggle/working/circo/data/stats_' + dpt_id_selected + '.csv'
df_stats_circo_selected = pd.read_csv(file_path_circo_stats,low_memory=False)

df_stats_circo_demo = df_stats_circo_selected[['Nom de la circonscription','Inscrit_22','pop_légal_19','pop_légal_13','tvar_pop','pop_pole_aav','pop_cour_aav','pop_horsaav','pop_urb','pop_rur_periu','pop_rur_non_periu','age_moyen','dec90','dec75','dec50','dec25','dec10']]
df_stats_circo_acti = df_stats_circo_selected[['Nom de la circonscription','actemp','actcho','inactret','inactetu','inactm14','inactaut','actemp_hom','actcho_hom','inactret_hom','inactetu_hom','inactm14_hom','inactaut_hom','actemp_fem','actcho_fem','inactret_fem','inactetu_fem','inactm14_fem','inactaut_fem']]
df_stats_circo_qualif = df_stats_circo_selected[['Nom de la circonscription','actdip_PEU','actdip_CAP','actdip_BAC','actdip_BAC2','actdip_BAC3','actdip_BAC5','actdip_BAC3P']]
df_stats_circo_secteur = df_stats_circo_selected[['Nom de la circonscription','act_agr','act_art','act_cad','act_int','act_emp','act_ouv','act_cho']]
df_stats_circo_log = df_stats_circo_selected[['Nom de la circonscription','log_res','log_sec','log_vac','proprio','locatai','gratuit','maison','ach90','mfuel']]
df_stats_circo_fam = df_stats_circo_selected[['Nom de la circonscription','men_seul','men_coupae','men_coupse','men_monop','men_sfam','men_seul_com','men_coupse_com','men_coupae_com','men_monop_com','men_complexe_com']]
df_stats_circo_mob = df_stats_circo_selected[['Nom de la circonscription','iranr_com','iranr_dep','iranr_fra','iranr_etr','mobresid','ilt_com','ilt_dep','ilt_fra','ilt_etr','mobtrav','modtrans_aucun','modtrans_pied','modtrans_velo','modtrans_moto','modtrans_voit','modtrans_commun']]
df_stats_circo_niv_vie = df_stats_circo_selected[['Nom de la circonscription','tx_pauvrete60_diff','nivvie_median_diff','part_pauvres_diff','part_modestes_diff','part_medians_diff','part_plutot_aises_diff','part_aises_diff','D1_diff','D9_diff','rpt_D9_D1_diff','tx_pauvrete60_diff_trageRF1','tx_pauvrete60_diff_trageRF2','tx_pauvrete60_diff_trageRF3','tx_pauvrete60_diff_trageRF4','tx_pauvrete60_diff_trageRF5','tx_pauvrete60_diff_trageRF6']]
df_stats_circo_rev = df_stats_circo_selected[['Nom de la circonscription','PACT','PPEN','PPAT','PPSOC','PIMPOT']]
df_stats_circo_autres = df_stats_circo_selected[['Nom de la circonscription','acc_ecole','acc_college','acc_lycee','acc_medecin','acc_dentiste','acc_pharmacie','part_eloig']]

with st.expander("Statistiques descriptives:"):
    tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9, tab10 = st.tabs(["Démographie", "Activité/inactivité:", "Qualification","Secteurs","Logement","Famille","Mobilité","Niveau de vie","Revenus","Autres" ])
    
    with tab1:
        st.dataframe(df_stats_circo_demo,hide_index=True)
        st.dataframe(df_stats_def_demo,hide_index=True)
    with tab2:  
        st.dataframe(df_stats_circo_acti,hide_index=True)
        st.dataframe(df_stats_def_acti,hide_index=True)
    with tab3: 
        st.dataframe(df_stats_circo_qualif,hide_index=True)
        st.dataframe(df_stats_def_qualif,hide_index=True)
    with tab4:
        st.dataframe(df_stats_circo_secteur,hide_index=True)
        st.dataframe(df_stats_def_secteur,hide_index=True)
    with tab5:  
        st.dataframe(df_stats_circo_log,hide_index=True)
        st.dataframe(df_stats_def_log,hide_index=True)
    with tab6:
        st.dataframe(df_stats_circo_fam,hide_index=True)
        st.dataframe(df_stats_def_fam,hide_index=True)
    with tab7: 
        st.dataframe(df_stats_circo_mob,hide_index=True)
        st.dataframe(df_stats_def_mob,hide_index=True)
    with tab8: 
        st.dataframe(df_stats_circo_niv_vie,hide_index=True)
        st.dataframe(df_stats_def_niv_vie,hide_index=True)
    with tab9:  
        st.dataframe(df_stats_circo_rev,hide_index=True)
        st.dataframe(df_stats_def_rev,hide_index=True)
    with tab10:
        st.dataframe(df_stats_circo_autres,hide_index=True)
        st.dataframe(df_stats_def_autres,hide_index=True)

# c.Résultats des circonscriptions du département sélectionné
file_path_circo_resultats = '/kaggle/working/circo/data/resultats_' + dpt_id_selected + '.csv'
df_resultats_circo_selected = pd.read_csv(file_path_circo_resultats,low_memory=False)

    # i. Stats descriptives
circo_resultats_overview = df_resultats_circo_selected[['Libellé circonscription législative', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
circo_resultats_overview = circo_resultats_overview.drop_duplicates()
data_container3 = st.container()
with data_container3:
    st.write("Circonscriptions - Elections européennes:")
    st.dataframe(circo_resultats_overview,hide_index=True)

    # ii. Résultats (top10)
circo_resultats_details = df_resultats_circo_selected[['id_circo','Libellé circonscription législative','indicateur','valeur']]
circo_resultats_details['id_liste'] = circo_resultats_details['indicateur'].str[-2:]
circo_resultats_details['id_liste'] = circo_resultats_details['id_liste'].str.strip()

circo_resultats_details['indicateur'] = circo_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
circo_resultats_details['indicateur']= circo_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

#circo_resultats_details

with st.expander("Circonscriptions - Résultats européennes (top10):"):
    groups = circo_resultats_details.groupby('id_circo')
    for name,group in groups:
            tmp_details_circo = group.pivot(index = ['id_circo','id_liste'], columns='indicateur', values='valeur')
            tmp_details_circo = pd.DataFrame(tmp_details_circo.to_records())
            tmp_details_circo["Voix"] = tmp_details_circo["Voix"].astype(int)
            tmp_details_circo = tmp_details_circo.nlargest(10, 'Voix')
            tmp_details_circo = tmp_details_circo[['id_liste','Libellé de liste','Nuance liste', 'Voix','% Voix/exprimés','% Voix/inscrits']]
            st.write(name)
            st.dataframe(tmp_details_circo,hide_index=True)   

# d.Liste des circonscriptions du département sélectionné
df_circo = df[df['id_dep'] == dpt_id_selected].drop(columns=['codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv']).drop_duplicates()
df_circo['id_circo'] = df_circo['id_circo'].astype(str)
df_circo.sort_values(by='id_circo',inplace=True)
df_circo['circo_lib'] = df_circo['id_circo'].str.cat(df_circo['nomCirconscription'], sep = ' - ')

circo =  df_circo['circo_lib'].drop_duplicates().sort_values()
circo_selected = st.sidebar.selectbox('Sélection de la circonscription:', circo)
# ID circo
circo_id_selected = str(circo_selected).split(" - ")[0]

# 3.Bureaux de votes de la circonscription sélectionnée

# a.Carte
# Read the HTML content from the file
html_content2 = read_html_file('/kaggle/working/bv/map/map_' + circo_id_selected + '.html')
# Display the HTML content in Streamlit
map_container2 = st.container()
with map_container2:
    st.write("Bureaux de vote:")
    st.components.v1.html(html_content2,height=500)

# b.Résultats des circonscriptions du département sélectionné
file_path_bv_resultats = '/kaggle/working/bv/data/resultats_' + circo_id_selected + '.csv'
df_resultats_bv_selected = pd.read_csv(file_path_bv_resultats,low_memory=False)

    # i. Stats descriptives
bv_resultats_overview = df_resultats_bv_selected[['codeBureauVote','bv_lib', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
bv_resultats_overview = bv_resultats_overview.drop_duplicates()
data_container3 = st.container()
with data_container3:
    st.write("Bureaux de votes - Elections européennes:")
    st.dataframe(bv_resultats_overview,hide_index=True)

    # ii. Résultats (top10)
bv_resultats_details = df_resultats_bv_selected[['codeBureauVote','bv_lib','indicateur','valeur']]
bv_resultats_details['id_liste'] = bv_resultats_details['indicateur'].str[-2:]
bv_resultats_details['id_liste'] = bv_resultats_details['id_liste'].str.strip()

bv_resultats_details['indicateur'] = bv_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
bv_resultats_details['indicateur']= bv_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)



bv =  bv_resultats_details['codeBureauVote'].drop_duplicates().sort_values()
bv_selected = st.sidebar.selectbox('Sélection du bureau de vote:', bv)
# ID BV
bv_id_selected = bv_selected

df_bv = bv_resultats_details[bv_resultats_details['codeBureauVote'] == bv_id_selected]

tmp_details_bv = df_bv.pivot(index = ['codeBureauVote','id_liste'], columns='indicateur', values='valeur')
tmp_details_bv = pd.DataFrame(tmp_details_bv.to_records())
tmp_details_bv["Voix"] = tmp_details_bv["Voix"].astype(int)
tmp_details_bv = tmp_details_bv.nlargest(10, 'Voix')
tmp_details_bv = tmp_details_bv[['codeBureauVote','id_liste','Libellé de liste','Nuance liste', 'Voix','% Voix/exprimés','% Voix/inscrits']]


data_container4 = st.container()
with data_container4:
    st.write("Bureau de vote sélectionné - Résultats européennes (top10):")
    st.dataframe(tmp_details_bv,hide_index=True)


Overwriting app.py


In [49]:
!npm install localtunnel

m#########.........] | idealTree: timing idealTree Completed in 110ms
up to date, audited 23 packages in 753ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [50]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.38.149.222


In [51]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

your url is: https://better-carrots-dress.loca.lt
^C
